In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from fastai.vision import *
from fastai.metrics import accuracy
from fastai.basic_data import *
from skimage.util import montage
import pandas as pd
from torch import optim
import re
import json
import cv2
# import albumentations
from fastai.callbacks.hooks import num_features_model
from torch.nn import L1Loss

In [2]:
import numpy as np
import torch
import pandas as pd
import random
import string

In [3]:
model_name = 'efficientnet-b2'

In [ ]:
from efficientnet_pytorch import EfficientNet
model = EfficientNet.from_pretrained(model_name) 

Downloading: "http://storage.googleapis.com/public-models/efficientnet/efficientnet-b2-8bb594d6.pth" to /files2c/1106817/.cache/torch/checkpoints/efficientnet-b2-8bb594d6.pth


In [ ]:
np.random.seed(42)

In [ ]:
image_size = EfficientNet.get_image_size(model_name)

In [ ]:
src = (ImageList.from_folder(path='../../data/train/').split_by_rand_pct(0.2).label_from_folder())

In [ ]:
res = []
res.append(rotate(degrees=(-90,90), p=1))

In [ ]:
tfms=(res,[])

In [ ]:
data = (src.transform(tfms, size=image_size, resize_method=ResizeMethod.SQUISH).databunch(bs=32).normalize(imagenet_stats))

In [ ]:
data.show_batch(3,figsize=(9,9))

In [ ]:
list(children(model))[-2:]

In [ ]:
model.add_module('_fc',nn.Linear(1536, data.c))

In [ ]:
loss_func = LabelSmoothingCrossEntropy()

In [ ]:
RMSprop = partial(torch.optim.RMSprop)

In [ ]:
from torch.optim import Optimizer

In [ ]:
learn = Learner(data, model, loss_func=loss_func, opt_func=RMSprop, metrics=[accuracy,FBeta(beta=1,average='macro')]).to_fp16()

In [ ]:
learn.split([[learn.model._conv_stem, learn.model._bn0, learn.model._blocks[:8]],
             [learn.model._blocks[8:],learn.model._conv_head], 
             [learn.model._bn1,learn.model._fc]])
print("pls")

In [ ]:
lr=1e-3

In [ ]:
learn.fit_one_cycle(40,max_lr=slice(lr/100,lr), callbacks=[SaveModelCallback(learn, every='improvement', monitor='accuracy', name='best')])

In [ ]:
save_name = model_name+"_trained_for_40"
learn.save(save_name, return_path=True)